<a href="https://colab.research.google.com/github/federicOO1/LAB-IA/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import librerie


In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
os.chdir("/content/drive/MyDrive/4_Ortho_RGBIR")

In [18]:
class PotsdamDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.transform = transform
        self.image_paths = []
        self.world_file_paths = []

        # Leggi i percorsi delle immagini e dei file .tfw
        for file_name in os.listdir(data_folder):
            if file_name.endswith('.tif'):
                image_path = os.path.join(data_folder, file_name)
                world_file_path = os.path.join(data_folder, file_name.replace('.tif', '.tfw'))
                if os.path.exists(world_file_path):
                    self.image_paths.append(image_path)
                    self.world_file_paths.append(world_file_path)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        world_file_path = self.world_file_paths[idx]

        # Leggi l'immagine e il file .tfw
        image = Image.open(image_path)
        world_params = self.load_world_file(world_file_path)

        # Applica trasformazioni se definite
        if self.transform:
            image = self.transform(image)

        return image, world_params

    def load_world_file(world_file_path):
      with open(world_file_path, 'r') as f:
          lines = f.readlines()
          # I file .tfw contengono 6 parametri numerici su 6 righe separate
          # Leggi e restituisci i parametri di georeferenziazione come una tupla di float
          try:
              parameters = tuple(float(line.strip()) for line in lines if line.strip())
              if len(parameters) == 6:
                  return parameters
              else:
                  raise ValueError("Il file .tfw non contiene 6 parametri.")
          except Exception as e:
              print(f"Errore durante la lettura dei parametri di georeferenziazione: {str(e)}")
              return None


In [19]:
# Definisci il percorso della cartella contenente i dati .tif e .tfw
data_folder = '/content/drive/My Drive/4_Ortho_RGBIR'

# Crea un'istanza del dataset
dataset = PotsdamDataset(data_folder, transform=transforms.ToTensor())

# Crea un DataLoader
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for batch_idx, (images, world_params) in enumerate(dataloader):
    print(f'Batch {batch_idx + 1} - Numero di immagini nel batch: {images.size(0)}')

    # Visualizza le immagini nel batch
    for i in range(images.size(0)):
        image = transforms.ToPILImage()(images[i])  # Converte il tensore in immagine PIL
        plt.imshow(image)
        plt.title(f'Immagine {i + 1} - Parametri di Georeferenziazione: {world_params[i]}')
        plt.axis('off')
        plt.show()


UnidentifiedImageError: ignored

In [ ]:



# Lista di percorsi di immagini nel tuo dataset
image_paths = ['path/to/image1.tif', 'path/to/image2.tif', 'path/to/image3.tif']

# Numero di righe e colonne nella griglia
num_rows = 1
num_cols = len(image_paths)

# Crea una griglia di immagini
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5))

for i, image_path in enumerate(image_paths):
    image = Image.open(image_path)
    axes[i].imshow(image)
    axes[i].set_title(f'Immagine {i+1}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()
